# Simplified Blackjack
In this notebook, I will attempt to code a game of simplified blackjack (assumptions below). Afterwards, there are some walkthrough tutorial exercises which were completed after my attempt for learning purposes. 

Game starts with 2 cards each, and one of the dealer's cards is face down. 

Player then chooses to stay (don't receive a card) or hit (receive a card) repeatedly until they want to stay. If player goes over 21, they lose. If the player gets 21 exactly, they win.
Otherwise: 

Dealer reveals the hidden card/hits until they beat the player (whilst 21 or under) or go bust. 

Scarios with Ace:
- Start with value = 11. As cards are drawn by, check for bust. If bust happens and Ace exists, change its value to 1 and continue to draw or stay





## Game Play
To play a hand of Blackjack the following steps must be followed:
1. Create a deck of 52 cards
2. Shuffle the deck
3. Ask the Player for their bet
4. Make sure that the Player's bet does not exceed their available chips
5. Deal two cards to the Dealer and two cards to the Player
6. Show only one of the Dealer's cards, the other remains hidden
7. Show both of the Player's cards
8. Ask the Player if they wish to Hit, and take another card
9. If the Player's hand doesn't Bust (go over 21), ask if they'd like to Hit again.
10. If a Player Stands, play the Dealer's hand. The dealer will always Hit until the Dealer's value meets or exceeds 17
11. Determine the winner and adjust the Player's chips accordingly
12. Ask the Player if they'd like to play again

#### Assumptions 

In this game of simplified blackjack, we are ignoring double down, insurance or Split

In [12]:
import random

In [13]:
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 
            'Nine':9, 'Ace':11}

class Card:
    
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        if self.rank in ['Ten', 'Jack', 'Queen', 'King']:
            self.value = 10
        else:
            self.value = values[rank]
            
    def __str__(self):
        return self.rank+' of '+self.suit+f' (value {self.value})'
    
    
class Deck:
    
    def __init__(self):
        self.cards = []
        for s in suits:
            for r in ranks:
                self.cards.append(Card(s,r))
        cards = random.shuffle(self.cards)
                
    def __str__(self):
        return f'a deck of {len(self.cards)} cards'
        
    def deal_one(self):
        card = self.cards.pop(0) # top of the deck
        return card

class Betting_account:
    def __init__(self, starting_amount):
        self.balance = starting_amount
        
    def new_bet(self,amount):
        self.bet_amount = amount
        
    def win_bet(self):
        self.balance += self.bet_amount
        
    def lose_bet(self):
        self.balance -= self.bet_amount
        
class Game_values:
    def __init__(self, player_value = 0, dealer_value = 0):
        self.player_value = player_value
        self.dealer_value = dealer_value
    
    def reset_values(self):
        self.player_value = 0
        self.dealer_value = 0

In [14]:
test_bet_account = Betting_account(100)
test_bet_account.balance

100

In [15]:
test_bet_account.new_bet(10)
test_bet_account.lose_bet()
test_bet_account.balance

90

In [16]:
def update_values(hide_dealer = False):
    global game
    game = Game_values() # New instance of game values
    # Player
    game.player_value = sum([i.value for i in player_hand])
    print("Player has:")
    for i in player_hand:
        print(f"- {i}")
    print(f"for a total value of {game.player_value}")

    # dealer
    if hide_dealer == True:
        game.dealer_value = dealer_hand[0].value
        print(f"Dealer has: {dealer_hand[0]} and 1 unknown card")
    else:
        game.dealer_value = sum([i.value for i in dealer_hand])
        print("Dealer has:")
        for i in dealer_hand:
            print(f"- {i}")
        print(f"for a total value of {game.dealer_value}")
    
    
def adjust_value_of_ace(list_of_cards):
    '''
    Returns True if an ace was adjusted to score 1
    Returns False if player/dealer is bust
    '''
    values = [i.value for i in list_of_cards]
    if sum(values) > 21 and 11 in values:
        # eligible for adjustment
        for index in range(len(values)):
            if values[index] == 11:
                list_of_cards[index].value = 1
                return True #ensures only 1 Ace is adjusted 
    elif sum(values) > 21:
        return False

    
def check_win_or_loss():
    player_value = sum([i.value for i in player_hand])
    dealer_value = sum([i.value for i in dealer_hand])
    if player_value == 21:
        print("Player's won!")
        myaccount.win_bet()
        return True
    elif player_value > 21:
        print("Player's gone bust!")
        myaccount.lose_bet()
        return True
    elif dealer_value > 21:
        print("Dealer's gone bust!")
        myaccount.win_bet()
        return True
    elif dealer_value > player_value:
        print("Dealer wins!")
        myaccount.lose_bet()
        return True
    return False

In [21]:
myaccount = Betting_account(1000)

In [19]:

while True:
    mydeck = Deck()

    # Ask player to place bet
    while True:
        try:
            bet_input = int(input('How much would you like to bet? '))
            myaccount.new_bet(bet_input)
        except:
            print('enter a valid integer')
    # Check that player has enough to bet this amount
        else: # executes if there is no exception
            if myaccount.balance < bet_input:
                print(f'Not enough funds! You only have £{myaccount.balance} available')
                continue
            elif myaccount.balance >= bet_input:
                myaccount.new_bet = bet_input
                break

    print(f"Current bet is £{myaccount.new_bet}. Let's play!")

    player_hand, dealer_hand = [], []

    for i in range(2):
        player_hand.append(mydeck.deal_one())
        dealer_hand.append(mydeck.deal_one())

    # In case 2 Aces are drawn, 1 will be adjusted to have value of 1 to prevent bust
    adjust_value_of_ace(player_hand) 
    adjust_value_of_ace(dealer_hand)
    # Reveal cards dealt to player and 1 of the cards dealt to dealer
    update_values(hide_dealer = True)
    
        
    player_loss = False
    while True:
        hit_or_stay = input("'H' to hit, 'S' to stay: ")
        if hit_or_stay == 'H':
            player_hand.append(mydeck.deal_one())
            adjust_value_of_ace(player_hand)
            update_values(hide_dealer = True)
            if check_win_or_loss():
                player_loss = True
                break
        elif hit_or_stay == 'S':
            break
            
    if player_loss == False:
        # reveal hidden card and check for win
        adjust_value_of_ace(dealer_hand)
        update_values(hide_dealer = False)
        if check_win_or_loss():
            break
        #if no win yet, then dealer hits until he beats player or busts 
        while True:
            dealer_hand.append(mydeck.deal_one())
            adjust_value_of_ace(dealer_hand)
            update_values(hide_dealer = False)
            if check_win_or_loss():
                break
        break

    else:
        break   


Current bet is £100. Let's play!
Player has:
- Six of Diamonds (value 6)
- Five of Spades (value 5)
for a total value of 11
Dealer has: Seven of Diamonds (value 7) and 1 unknown card
Player has:
- Six of Diamonds (value 6)
- Five of Spades (value 5)
- Ten of Diamonds (value 10)
for a total value of 21
Dealer has: Seven of Diamonds (value 7) and 1 unknown card
Player's won!


In [20]:
myaccount.balance

1100

### Create a card class

Each card has 1 of 4 suits and 1 of 13 ranks. Depending on rank, the card has a value. Jack, Queen and King all have value of 10 for this game. Ace can be a 1 or 11 depending on what the player prefers. Makes most sense to start with 11, then if the score is over 21, downgrade Ace to 1 to prevent going bust. 

## Playing Cards
A standard deck of playing cards has four suits (Hearts, Diamonds, Spades and Clubs) and thirteen ranks (2 through 10, then the face cards Jack, Queen, King and Ace) for a total of 52 cards per deck. Jacks, Queens and Kings all have a rank of 10. Aces have a rank of either 11 or 1 as needed to reach 21 without busting. As a starting point in your program, you may want to assign variables to store a list of suits, ranks, and then use a dictionary to map ranks to values.

## The Game
### Imports and Global Variables
** Step 1: Import the random module. This will be used to shuffle the deck prior to dealing. Then, declare variables to store suits, ranks and values. You can develop your own system, or copy ours below. Finally, declare a Boolean value to be used to control <code>while</code> loops. This is a common practice used to control the flow of the game.**

    suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
    ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
    values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10, 'Jack':10,
             'Queen':10, 'King':10, 'Ace':11}

In [ ]:
import random

suits = pass
ranks = pass
values = pass

playing = True

### Class Definitions
Consider making a Card class where each Card object has a suit and a rank, then a Deck class to hold all 52 Card objects, and can be shuffled, and finally a Hand class that holds those Cards that have been dealt to each player from the Deck.

**Step 2: Create a Card Class**<br>
A Card object really only needs two attributes: suit and rank. You might add an attribute for "value" - we chose to handle value later when developing our Hand class.<br>In addition to the Card's \_\_init\_\_ method, consider adding a \_\_str\_\_ method that, when asked to print a Card, returns a string in the form "Two of Hearts"

In [ ]:
class Card:
    
    def __init__(self):
        pass
    
    def __str__(self):
        pass

**Step 3: Create a Deck Class**<br>
Here we might store 52 card objects in a list that can later be shuffled. First, though, we need to *instantiate* all 52 unique card objects and add them to our list. So long as the Card class definition appears in our code, we can build Card objects inside our Deck \_\_init\_\_ method. Consider iterating over sequences of suits and ranks to build out each card. This might appear inside a Deck class \_\_init\_\_ method:

    for suit in suits:
        for rank in ranks:

In addition to an \_\_init\_\_ method we'll want to add methods to shuffle our deck, and to deal out cards during gameplay.<br><br>
OPTIONAL: We may never need to print the contents of the deck during gameplay, but having the ability to see the cards inside it may help troubleshoot any problems that occur during development. With this in mind, consider adding a \_\_str\_\_ method to the class definition.

In [ ]:
class Deck:
    
    def __init__(self):
        self.deck = []  # start with an empty list
        for suit in suits:
            for rank in ranks:
                pass
    
    def __str__(self):
        pass

    def shuffle(self):
        random.shuffle(self.deck)
        
    def deal(self):
        pass

TESTING: Just to see that everything works so far, let's see what our Deck looks like!

In [ ]:
test_deck = Deck()
print(test_deck)

Great! Now let's move on to our Hand class.

**Step 4: Create a Hand Class**<br>
In addition to holding Card objects dealt from the Deck, the Hand class may be used to calculate the value of those cards using the values dictionary defined above. It may also need to adjust for the value of Aces when appropriate.

In [ ]:
class Hand:
    def __init__(self):
        self.cards = []  # start with an empty list as we did in the Deck class
        self.value = 0   # start with zero value
        self.aces = 0    # add an attribute to keep track of aces
    
    def add_card(self,card):
        pass
    
    def adjust_for_ace(self):
        pass

**Step 5: Create a Chips Class**<br>
In addition to decks of cards and hands, we need to keep track of a Player's starting chips, bets, and ongoing winnings. This could be done using global variables, but in the spirit of object oriented programming, let's make a Chips class instead!

In [ ]:
class Chips:
    
    def __init__(self):
        self.total = 100  # This can be set to a default value or supplied by a user input
        self.bet = 0
        
    def win_bet(self):
        pass
    
    def lose_bet(self):
        pass

### Function Defintions
A lot of steps are going to be repetitive. That's where functions come in! The following steps are guidelines - add or remove functions as needed in your own program.

**Step 6: Write a function for taking bets**<br>
Since we're asking the user for an integer value, this would be a good place to use <code>try</code>/<code>except</code>. Remember to check that a Player's bet can be covered by their available chips.

In [ ]:
def take_bet():
    
    pass

**Step 7: Write a function for taking hits**<br>
Either player can take hits until they bust. This function will be called during gameplay anytime a Player requests a hit, or a Dealer's hand is less than 17. It should take in Deck and Hand objects as arguments, and deal one card off the deck and add it to the Hand. You may want it to check for aces in the event that a player's hand exceeds 21.

In [ ]:
def hit(deck,hand):
    
    pass

**Step 8: Write a function prompting the Player to Hit or Stand**<br>
This function should accept the deck and the player's hand as arguments, and assign playing as a global variable.<br>
If the Player Hits, employ the hit() function above. If the Player Stands, set the playing variable to False - this will control the behavior of a <code>while</code> loop later on in our code.

In [ ]:
def hit_or_stand(deck,hand):
    global playing  # to control an upcoming while loop
    
    pass

**Step 9: Write functions to display cards**<br>
When the game starts, and after each time Player takes a card, the dealer's first card is hidden and all of Player's cards are visible. At the end of the hand all cards are shown, and you may want to show each hand's total value. Write a function for each of these scenarios.

In [ ]:
def show_some(player,dealer):
    
    pass
    
def show_all(player,dealer):
    
    pass

**Step 10: Write functions to handle end of game scenarios**<br>
Remember to pass player's hand, dealer's hand and chips as needed.

In [ ]:
def player_busts():
    pass

def player_wins():
    pass

def dealer_busts():
    pass
    
def dealer_wins():
    pass
    
def push():
    pass

### And now on to the game!!

In [ ]:
while True:
    # Print an opening statement

    
    # Create & shuffle the deck, deal two cards to each player

    
        
    # Set up the Player's chips
    
    
    # Prompt the Player for their bet

    
    # Show cards (but keep one dealer card hidden)

    
    while playing:  # recall this variable from our hit_or_stand function
        
        # Prompt for Player to Hit or Stand
        
        
        # Show cards (but keep one dealer card hidden)
 
        
        # If player's hand exceeds 21, run player_busts() and break out of loop
        

            break

    # If Player hasn't busted, play Dealer's hand until Dealer reaches 17
    
    
        # Show all cards
    
        # Run different winning scenarios
        
    
    # Inform Player of their chips total 
    
    # Ask to play again

        break

And that's it! Remember, these steps may differ significantly from your own solution. That's OK! Keep working on different sections of your program until you get the desired results. It takes a lot of time and patience! As always, feel free to post questions and comments to the QA Forums.
# Good job!